In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

visit_origin = os.environ.get("SCHEDVIEW_VISIT_ORIGIN", "lsstcam")
day_obs = int(
    os.environ.get(
        "SCHEDVIEW_DAY_OBS",
        (datetime.date.today() - datetime.timedelta(days=1)).isoformat()
    ).replace("-", "")
)

"""
# To render this notebook to html, in the current directory with the current python environment

export ACCESS_TOKEN_FILE=${HOME}/.lsst/usdf_access_token
export SCHEDVIEW_VISIT_ORIGIN=lsstcam
export SCHEDVIEW_DAY_OBS='20251108'
jupyter nbconvert \
    --to html \
    --execute \
    --no-input \
    --ExecutePreprocessor.kernel_name=python3 \
    --ExecutePreprocessor.startup_timeout=3600 \
    --ExecutePreprocessor.timeout=3600 \
    scheduler-nightsum.ipynb

# The public USDF page served at
#    https://usdf-rsp-int.slac.stanford.edu/schedview-static-pages/nightsum/lsstcam/2025/11/08/nightsum_2025-11-08.html
# is found at
#    /sdf/data/rubin/shared/scheduler/reports/nightsum/lsstcam/2025/11/08/nightsum_2025-11-08.html
"""
pass

In [ ]:
if False:
    visit_origin = 'lsstcam'
    day_obs = 20260116

In [ ]:
if isinstance(day_obs, str):
    day_obs = int(day_obs)

In [ ]:
import os
import sys

#sched_source = 'env'
sched_source = 'shared'
#sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.4.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.22.0")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.6.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.22.0a2")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_nights-0.11.0")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/uranography")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_nights")
    case _:
        # Use whatever is in the kernel python environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
import datetime
import os
import sqlite3
import sys
from collections import defaultdict
from functools import partial
from pathlib import Path

import astropy
import astropy.units as u
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.transform
import cartopy
import colorcet
import healpy
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skyproj
from erfa import ErfaWarning
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time
from astropy.visualization import ZScaleInterval
from IPython.display import HTML, display, Markdown, HTML
from lsst.resources import ResourcePath

In [ ]:
running_on_ts = os.getenv("USER").startswith("bot-noteburst")

In [ ]:
usdf_sim_data_dir = "/sdf/data/rubin/shared/rubin_sim_data"
if os.path.exists(usdf_sim_data_dir):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
import schedview.collect.consdb
import schedview.collect.nightreport
import schedview.collect.timeline
import rubin_scheduler
import rubin_scheduler.site_models
import rubin_scheduler.utils
import schedview.compute.astro
import schedview.compute.visits
import schedview.compute.nightreport
import schedview.plot.survey_skyproj
import schedview.plot.visitmap
import schedview.plot.nightreport
import schedview.collect.visits
import rubin_sim.sim_archive
import uranography
from schedview.compute.maf import compute_metric
from rubin_sim.data import get_baseline
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from uranography.api import Planisphere, make_zscale_linear_cmap
from schedview import DayObs
from schedview.plot import mpl_fig_to_html
from rubin_scheduler.utils.consdb import KNOWN_INSTRUMENTS
from schedview.plot.survey_skyproj import map_hpix_in_laea_and_mcbryde, map_metric_in_laea_and_mcbryde
from rubin_nights.reference_values import SCIENCE_PROGRAMS

In [ ]:
if not Path('/sdf/group/rubin/web_data/sim-data').exists():
    schedview.collect.visits.OPSIMDB_TEMPLATE = str(
        Path.home().joinpath("Data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db")
    )

In [ ]:
# Validate the input
import re

assert visit_origin in KNOWN_INSTRUMENTS or visit_origin == 'baseline' or re.match(r"^\d+\.\d+$", visit_origin) is not None

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
%matplotlib inline

In [ ]:
ORIGIN_TELESCOPE = defaultdict(
    np.array(['Simonyi']).item,
    {'latiss': 'AuxTel'}
)

In [ ]:
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"
telescope = ORIGIN_TELESCOPE[visit_origin]
from_opsim = visit_origin not in KNOWN_INSTRUMENTS
use_matplotlib = True
science_programs_to_include = SCIENCE_PROGRAMS
telescope = "AuxTel" if visit_origin.lower()=="latiss" else "Simonyi"

In [ ]:
if from_opsim:
    display(Markdown(f"# Scheduler-oriented night summary of {day_obs} from baseline {visit_origin}"))
else:
    display(Markdown(f"# Scheduler-oriented night summary of {visit_origin} on {day_obs}"))

This report is a summary of a night of observing designed to understand scheduler behavior during the night, identify scheduler problems, and otherwise monitor scheduler performance.

In [ ]:
day_obs = DayObs.from_date(day_obs)
observatory.mjd = day_obs.mjd + 1 - observatory.location.lon.deg/360 ;# The approximate middle of the night

In [ ]:
visits = schedview.collect.visits.read_visits(day_obs, visit_origin, stackers = schedview.collect.visits.NIGHT_STACKERS)

In [ ]:
if 'can_see_sky' in visits.columns:
    visits = visits.loc[visits.can_see_sky.fillna(True).astype(bool), :]

In [ ]:
if not from_opsim:
    night_reports = schedview.collect.nightreport.get_night_report(day_obs, telescope)
    best_version_of_night_report =  schedview.compute.nightreport.best_night_report(night_reports)
    night_report_markdown = schedview.plot.nightreport.night_report_markdown(best_version_of_night_report)
    display(Markdown(night_report_markdown))

## Sun and Moon

### Events

In [ ]:
night_events = schedview.compute.astro.night_events(day_obs.date)
night_events

Modified Julian Date (MJD) is in units of days (UTC).

Local Sidereal Time (LST) is in units of degrees.

In [ ]:
# Refine the mjd of the middle of the night, unlikely to be important.
observatory.mjd = night_events.loc["night_middle", "MJD"]

### Positions at local solar midnight

In [ ]:
body_positions = schedview.compute.astro.compute_sun_moon_positions(observatory)
body_positions

RA, dec, alt, and az are all in units of degrees.

## Conditions and statistics

### Numbers of exposures, and gaps between them

In [ ]:
survey_visits = visits.loc[visits['science_program'].isin(science_programs_to_include), :] if visit_origin == 'lsstcam' else visits

In [ ]:
if len(survey_visits):
    survey_visit_summary = schedview.compute.visits.compute_survey_visit_summary(
        survey_visits,
        night_events.loc["sun_n12_setting", "MJD"],
        night_events.loc["sun_n12_rising", "MJD"],
    )

    survey_visit_table = schedview.plot.create_survey_visit_summary_table(survey_visit_summary)
    display(HTML(survey_visit_table))
else:
    print("No survey visits")

In [ ]:
if len(visits):
    overhead_summary = schedview.compute.visits.compute_overhead_summary(
        visits,
        night_events.loc["sun_n12_setting", "MJD"],
        night_events.loc["sun_n12_rising", "MJD"],
    )
    summary_table = schedview.plot.create_overhead_summary_table(overhead_summary)
    display(HTML(summary_table))
else:
    print("No visits")

## Value timelines

Select a value to plot in the subplot on the right using the dropdown at the top of that subplot.

In [ ]:
if from_opsim:
    if len(visits):
        fig = schedview.plot.plot_visit_param_vs_time(visits, 'airmass', show_column_selector=True)
        bokeh.io.show(fig)
    else:
        print("No visits")
else:
    user_param_plot_kwargs = {}

    timeline_data = await schedview.collect.timeline.collect_timeline_data(
            day_obs,
            sal_indexes=schedview.collect.SAL_INDEX_GUESSES[visit_origin],
            telescope=telescope,
            log_messages=False,
            scheduler_dependencies=True,
            scheduler_configuration=True,
            scheduler_snapshots=True,
            block_status=False,
        )
    timeline_data["model_sky"] = schedview.compute.astro.get_median_model_sky(day_obs)
    timeline_data['sun'] = night_events

    try:
        nominal_visits = rubin_sim.sim_archive.sim_archive.fetch_latest_prenight_sim_for_nights(
            str(day_obs),
            str(day_obs),
            max_simulation_age=1 + Time.now().mjd - day_obs.mjd,
            stackers=schedview.collect.visits.NIGHT_STACKERS,
            telescope=telescope,
        )
    except:
        nominal_visits = None

    if nominal_visits is not None:
        nominal_visits = nominal_visits.loc[(nominal_visits.observationStartMJD > day_obs.start.mjd) & (nominal_visits.observationStartMJD < day_obs.end.mjd), :]

    if nominal_visits is not None and len(nominal_visits)>0:
        timeline_data['visits'] = pd.concat([
            visits.assign(label='Completed'),
            nominal_visits.assign(
                label='Nominal',
            )
        ])
        user_param_plot_kwargs['marker_transform'] = bokeh.models.CategoricalMarkerMapper(factors=['Nominal', 'Completed'], markers=['circle', 'asterisk'])
    else:
        timeline_data['visits'] = visits
    
    p = schedview.plot.make_timeline_scatterplots(user_param_plot_kwargs=user_param_plot_kwargs, **timeline_data)
    bokeh.io.show(p)

## Visit map

In [ ]:
if len(visits):
    visits = schedview.compute.visits.add_coords_tuple(visits)

In [ ]:
if len(visits):
    vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
        visits=visits,
        night_date=day_obs.date,
        timezone=timezone,
        observatory=observatory,
    )
    bokeh.io.show(vmap)
else:
    print("No visits")

The above plots show the visits collected during the night in two different representations, modeled after physical observing tools.

 - The [Orthographic projection](https://en.wikipedia.org/wiki/Orthographic_projection) on the left is intended to be used as a virtual [armillary sphere](https://en.wikipedia.org/wiki/Armillary_sphere): a spherical representation of the sky whose orientation with respect to the horizon can be shown as a function of time. The time for which the orientation applies is set by the "Date and time (UTC)" slider, and the orientation from which the user views the figure is controlled by the "center alt" and "center az" sliders, which set the coordinates of the center of the figure. The figure is always oriented so zenith us "up" on the screen. The orthogrophic projection is not an equal-area projection, but playing with the sliders is a helpful way to inform a human's spatial reasoning in three dimensions.
 - The [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) on the left, centered at the south celestial pole, is intended to be used as a virtual [planisphere](https://en.wikipedia.org/wiki/Planisphere), with the "Date and time (UTC)" slider setting the time. R.A. increases counterclockwise. The projection used is equal area, but highly distorted near the north celestial pole (outside the LSST footprint). This is a particularly helpful representation for planning observing, because changes in time in relevant features are simple rotations, without alterations in distortion, and there are no discontinuities anywhere in the footprint at any time of year.

Both plots show the footprints of camera pointing taken up to the time set by the MJD slider, with the most recent three pointings outlined in cyan:

 - <span style='background-color:#1600ea'>&nbsp;&nbsp;&nbsp;</span><span style='color:#1600ea'> blue</span>: u band
 - <span style='background-color:#31de1f'>&nbsp;&nbsp;&nbsp;</span><span style='color:#31de1f'> green</span>: g band
 - <span style='background-color:#b52626'>&nbsp;&nbsp;&nbsp;</span><span style='color:#b52626'> red</span>: r band
 - <span style='background-color:#370201'>&nbsp;&nbsp;&nbsp;</span><span style='color:#370201'> brown</span>: i band
 - <span style='background-color:#ba52ff'>&nbsp;&nbsp;&nbsp;</span><span style='color:#ba52ff'> purple</span>: z band
 - <span style='background-color:#61a2b3'>&nbsp;&nbsp;&nbsp;</span><span style='color:#61a2b3'> teal</span>: y band
   
Both plots have the following additional annotations:
 - The thick gray lines show the boundries of different areas of the LSST footprint.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
   - The sun moves along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) such that it makes a full revolution in one year.
   - The moon moves roughly (within 5.14°) along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure), completing a full revolution in one [sidereal month](https://en.wikipedia.org/wiki/Lunar_month#Sidereal_month) (a bit over 27 days), about 14° per day.
 - The blue line (oval) shows the plane of the Milky Way.
 - The black line shows the horizon at the time set by the MJD slider.
 - The red line shows a zenith distince of 70° (airmass=2.9) at the time set by the MJD slider.

## Night visit metric maps

In all of the maps that follow, annotations are similar to those of the "Visit Map" above:

 - Black outlines show the camera footprint of each exposure taken on this night.
 - The orange disk shows the coordinates of the moon.
 - The brown disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.

### Map of numbers of visits on night

In [ ]:
if len(survey_visits) > 0 and telescope == "Simonyi":
    fig = map_metric_in_laea_and_mcbryde(
        survey_visits,
        maf.CountMetric(col='band'),
        schedview.plot.survey_skyproj.map_count_healpix,
        observatory,
        night_events,
        lon_0=night_events.loc['night_middle', 'LST'].item(),
        cmap='cividis_r',
        horizons=None,
        num_colors=8
    )
else:
    print(f"No survey visits from {telescope} on this night.")

### Map on number of bands on night

In [ ]:
if len(survey_visits) > 0 and telescope == "Simonyi":
    fig = map_metric_in_laea_and_mcbryde(
        survey_visits,
        maf.CountUniqueMetric(col='band'),
        schedview.plot.survey_skyproj.map_count_healpix,
        observatory,
        night_events,
        lon_0=night_events.loc['night_middle', 'LST'].item(),
        cmap='cividis_r',
        horizons=None,
        num_colors=5
    )
else:
    print(f"No survey visits from {telescope} on this night")

### Map of depth as inverse variance of noise (effective exposure time)

The following map shows accumulated inverse variance of visits completed on this night, a uniformly increasing measure of progress toward a target limiting coadd limiting magnitude. The inverse variance of the noise is increases linearly with the number of exposures for exposures of uniform depth, and total exposure time when noise is dominated by sky brightness, and so is sometimes scaled to a time under reference conditions and called the "effective exposure time." See DMTN-296: Calculations of Image and Catalog Depth for more details.

In [ ]:
if running_on_ts:
    print("Running on Times Square; skipping this figure to save time.")
elif telescope != "Simonyi":
    print("Not running on Simonyi, skipping")
else:
    map_html = ""
    with plt.ioff():
        for band in 'ugrizy':
            if band not in survey_visits.band.values:
                continue
                
            visits_in_band = survey_visits.loc[visits.band == band, :]
            if len(visits_in_band.loc[~ np.isnan(visits_in_band.eff_time_median.values.astype(float)), 'eff_time_median']) < 1:
                continue

            map_html = map_html + f"""
            <details><summary><b>Accumulated inverse noise variance (t<sub>eff</sub>) in {band} band</b></summary>
            """
    
            fig = map_metric_in_laea_and_mcbryde(
                visits_in_band,
                maf.SumMetric(col='eff_time_median'),
                schedview.plot.survey_skyproj.map_healpix,
                observatory,
                night_events,
                lon_0=night_events.loc['night_middle', 'LST'].item(),
                figsize=(15, 5),
                cmap='cividis_r',
                horizons=None)
    
            map_html = map_html + mpl_fig_to_html(fig)
            map_html = map_html + "</details>"
    
    if len(map_html)>0:
        display(HTML(map_html))

### Map of number of bands completed twice on night

In [ ]:
observed_twice_by_band = {}
if telescope == "Simonyi":
    for band in 'ugrizy':
        if band not in survey_visits.band.values:
            continue
    
        mb = maf.MetricBundle(
            maf.CountMetric(col='observationStartMJD'),
            maf.HealpixSlicer(nside=32, verbose=False)
        )
        compute_metric(survey_visits.set_index('band').loc[band, :], mb)
        observed_twice_by_band[band] = mb.metric_values.filled(0) >= 2

if len(observed_twice_by_band) > 0 and telescope == "Simonyi":
    bands_observed_twice = np.sum(list(observed_twice_by_band.values()), axis=0)
    # mask hpixels with value of 0
    bands_observed_twice = np.ma.masked_equal(bands_observed_twice, 0)
    
    with plt.ioff():
        fig = map_hpix_in_laea_and_mcbryde(
            schedview.plot.survey_skyproj.map_count_healpix,
            bands_observed_twice,
            observatory,
            night_events,
            lon_0=night_events.loc['night_middle', 'LST'].item(),
            cmap='cividis_r',
            horizons=None,
            num_colors=5
        )
else:
    fig = None

if fig is not None:
    display(fig)
else:
    if len(observed_twice_by_band) > 0:
        print("No bands observed twice")
    if telescope != "Simonyi":
        print(f"Not computed for {telescope}")

## Survey Progress

In all of the maps that follow, annotations are similar to those of the "Visit Map" above:

 - Black outlines show the camera footprint of each exposure taken on this night.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.

Maps begin "collapsed": to unfold them, click on the title of the desired projection.

In [ ]:
conditions = observatory.return_conditions()

In [ ]:
accumulated_visits = schedview.collect.visits.read_visits(day_obs, visit_origin, stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=10000)

In [ ]:
accumulated_survey_visits = accumulated_visits.loc[accumulated_visits['science_program'].isin(science_programs_to_include), :] if visit_origin == 'lsstcam' else visits

### Total numbers of visits accumulated so far

In [ ]:
if running_on_ts:
    print("Running on Times Square; skipping this figure to save time.")
elif telescope != "Simonyi":
    print(f"Skipping for {telescope}")
else:
    with plt.ioff():
        map_html = "<details><summary><b>Total visits accumumlated so far</b></summary>"
        
        fig = map_metric_in_laea_and_mcbryde(
            accumulated_survey_visits,
            maf.CountMetric(col='band'),
            schedview.plot.survey_skyproj.map_healpix,
            observatory,
            night_events,
            lon_0=night_events.loc['night_middle', 'LST'].item(),
            figsize=(15, 5),
            cmap='cividis_r',
            horizons=None,
        )
        
        map_html = map_html + mpl_fig_to_html(fig)
        map_html = map_html + "</details>"
    
        for band in 'ugrizy':
            if band not in accumulated_survey_visits.band.values:
                continue
                
            visits_in_band = accumulated_survey_visits.loc[accumulated_survey_visits.band == band, :]
            if len(visits_in_band.loc[~ np.isnan(visits_in_band.eff_time_median.values.astype(float)), 'eff_time_median']) < 1:
                continue
    
            map_html = map_html + f"""
            <details><summary><b>Total accumulated visits in {band} band</b></summary>
            """
    
            fig = map_metric_in_laea_and_mcbryde(
                visits_in_band,
                maf.CountMetric(col='band'),
                schedview.plot.survey_skyproj.map_healpix,
                observatory,
                night_events,
                lon_0=night_events.loc['night_middle', 'LST'].item(),
                figsize=(15, 5),
                cmap='cividis_r',
                horizons=None)
            map_html = map_html + mpl_fig_to_html(fig)
            map_html = map_html + "</details>"
    
        display(HTML(map_html))

### Map of depth (inverse variance of noise) for ideal coadd of visits accumulated so far

The following map shows accumulated inverse variance of completed visits, a uniformly increasing measure of progress toward a target limiting coadd limiting magnitude. The inverse variance of the noise is increases linearly with the number of exposures for exposures of uniform depth, and total exposure time when noise is dominated by sky brightness, and so is sometimes scaled to a time under reference conditions and called the "effective exposure time." See DMTN-296: Calculations of Image and Catalog Depth for more details.


In [ ]:
if running_on_ts:
    print("Running on Times Square; skipping this figure to save time.")
elif telescope != "Simonyi":
    print(f"Skipping for {telescope}")
else:
    with plt.ioff():
        map_html = ""
        for band in 'ugrizy':
            if band not in accumulated_survey_visits.band.values:
                continue
                
            visits_in_band = accumulated_survey_visits.loc[accumulated_survey_visits.band == band, :]
            if len(visits_in_band.loc[~ np.isnan(visits_in_band.eff_time_median.values.astype(float)), 'eff_time_median']) < 1:
                continue
    
            map_html = map_html + f"""
            <details><summary><b>Accumulated inverse noise variance (t<sub>eff</sub>) in {band} band</b></summary>
            """
    
            fig = map_metric_in_laea_and_mcbryde(
                visits_in_band,
                maf.SumMetric(col='eff_time_median'),
                schedview.plot.survey_skyproj.map_healpix,
                observatory,
                night_events,
                lon_0=night_events.loc['night_middle', 'LST'].item(),
                figsize=(15, 5),
                cmap='cividis_r',
                horizons=None)
            map_html = map_html + mpl_fig_to_html(fig)
            map_html = map_html + "</details>"
    
        display(HTML(map_html))

## DDF Cadence

In [ ]:
time_window_duration=90

In [ ]:
try:
    ddf_visits = schedview.collect.visits.read_ddf_visits(day_obs, visit_origin, num_nights=time_window_duration)
    if not from_opsim:
        ddf_visits = ddf_visits.loc[ddf_visits['science_program'].isin(science_programs_to_include), :]
except:
    ddf_visits = []

In [ ]:
if len(ddf_visits):
    nightly_ddf = schedview.compute.visits.accum_stats_by_target_band_night(ddf_visits, target_column='field_name')
    cadence_plots = schedview.plot.create_cadence_plot(
        nightly_ddf, day_obs.mjd - time_window_duration, day_obs.mjd, target_column='field_name'
    )
    bokeh.io.show(cadence_plots)
else:
    print("No DDF visits")

The y-axis (height of the vertical bars) represents the accumulated effective exposure time, t<sub>eff</sub> (as defined above) accumulated over all exposures on the field for the night, colored by filter.

## Histogram of gaps between exposures

In [ ]:
if len(visits):
    try:
        p1 = schedview.plot.create_overhead_histogram(visits)
        p2 = schedview.plot.plot_overhead_vs_slew_distance(visits)
        overhead_plots = bokeh.layouts.row([p1, p2])
        bokeh.io.show(overhead_plots)
    except Exception as e:
        print(e)
else:
    print("No visits")

"Overhead" is the time window (in seconds) between the end of the previous exposure and the end of the current exposure, minus the exposure time.

## Long gaps between exposures

In [ ]:
if len(visits):
    try:
        for time_column in 'obs_start', 'start_timestamp', 'observationStartDatetime64':
            if time_column in visits:
                break
        num_gaps = 10
        visits["previous_band"] = visits["band"].shift(1)
        long_gap_visits = (
            visits.sort_values("overhead", ascending=False)
            .query("overhead>30")
            .loc[:, [time_column, "overhead", "slewDistance", "band", "previous_band"]]
            .sort_values(time_column)
        )
        display(long_gap_visits)
    except Exception as e:
        print(e)
else:
    print("No visits")

These are the longest gaps between exposures that occur on the night, ordered by duration of the gaps.

Slew time and previous and current filters are noted, because filter changes are the greatest expected contributions to overhead.

## Table of exposures

In [ ]:
# The bokeh table widget and the Rubin RSP notebooks do not work together.
# If running on the Rubin RSP notebook aspect, set on_rsp=True.
# Otherwise, leave it as False
on_rsp = False

In [ ]:
if len(visits):
    for time_column in 'obs_start', 'observationStartDatetime64':
        if time_column in visits:
            break
    displayed_columns_maybe = [
        time_column,
        "fieldRA",
        "fieldDec",
        "band" if "band" in visits else "filter",
        "visitExposureTime",
        "fiveSigmaDepth",
        "eff_time_median",
        "skyBrightness",
        "seeingFwhmEff",
        "target_name",
        "observation_reason",
    ]
    displayed_columns = [c for c in displayed_columns_maybe if c in visits.columns]
    if on_rsp:
        with pd.option_context('display.max_rows', 2000):
            display(visits.loc[:,displayed_columns])
    else:
        schedview.plot.create_visit_table(visits, visible_column_names=displayed_columns, width=1024)
else:
    print("No visits")